In [ ]:
"""
Gloria's Vanilla Forward Folding RL implementation. Rn has no use, included for reference.
"""

In [95]:
import numpy as np
import pandas as pd
import sys
import gym
import io
import random

from gym_lattice.envs import Lattice2DEnv
from gym import spaces
from IPython.display import clear_output

## Defining Functions

In [14]:
def next_state(i,state,action):
    k = state-((4**(i-1)-1)/3)
    state1 = int((4**i-1)/3+4*(k-1)+(action+1))
    return state1

## RL with Q-Learning

In [158]:
seq = 'HPHHHPHHPH' # Our input sequence
action_space = spaces.Discrete(4) # Choose among [0, 1, 2 ,3]
env = Lattice2DEnv(seq)
states = int((4**len(seq)-1)/3)

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))
print("Total State Number {}".format(states))

Action Space Discrete(4)
State Space Box(21, 21)
Total State Number 349525


In [174]:
q_table = np.zeros([states, env.action_space.n])
alpha = 0.1
gamma = 0.6
episilon = 0.4
n = 60000

for x in range(n):
    # initialising
    env.reset()
    i = 1
    state = 0

    while i < len(seq):
        # chooses to explore or exploit
        if random.uniform(0,1) < episilon:
            action = env.action_space.sample() # explore
        else:
            action = np.argmax(q_table[state]) # exploit
        obs, reward, done, info = env.step(action) # executes action based on episilon

        # updates state number
        if info['is_trapped']:
            break
        elif i == info['chain_length']: # if collision happens
            continue # goes to start of while loop without updating state
        else:
            i = info['chain_length'] # updates chain length
            old_value = q_table[state, action]
            state2 = next_state(i,state,action) # gets next state
            next_max = np.max(q_table[state2])
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value
            state = state2
    episilon = episilon-(episilon/n)
    if x>=n-5:
        print(reward)


WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.
WARN: Your agent was trapped! Ending the episode.


In [173]:
print(episilon)
env.render()
print(reward)
print(info)

0.11036314257388745
  (Left)
*********************
*********************
*********************
*********************
*********************
*********************
*********************
*********************
**********H[0P*********
*********H[0H[0H[0*********
*********PH[0H[0*********
**********H[0P*********
*********************
*********************
*********************
*********************
*********************
*********************
*********************
*********************
*********************
4
{'chain_length': 10, 'seq_length': 10, 'collisions': 0, 'actions': ['L', 'U', 'R', 'U', 'R', 'D', 'D', 'D', 'L'], 'is_trapped': False, 'state_chain': OrderedDict([((0, 0), 'H'), ((-1, 0), 'P'), ((-1, 1), 'H'), ((0, 1), 'H'), ((0, 2), 'H'), ((1, 2), 'P'), ((1, 1), 'H'), ((1, 0), 'H'), ((1, -1), 'P'), ((0, -1), 'H')])}
